In [2]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import json
from sources import earwolf, rss

In [4]:
# Scrape data from sources and write to file

earwolf_eps = earwolf.scrape()
earwolf_eps.to_csv("data/earwolf_episodes.csv", index=False)

rss_eps = rss.scrape()
rss_eps.to_csv("data/rss_episodes.csv", index=False)

STARTING EARWOLF
Could not parse number for <li>Ep #UB1 - <a href="/episode/bonus-the-mysterious-secrets-of-uncle-berties-botanarium-episode-1/">Bonus! The Mysterious Secrets Of Uncle Bertie’s Botanarium – Episode 1</a> (<span>Jemaine Clement</span>)</li>
Could not parse number for <li>Ep #B3 - <a href="/episode/live-from-ucb-ny/">Live from UCB NY!</a> (<span>Zach Galifianakis</span>, <span>David Cross</span>, <span>Todd Barry</span>, <span>John Gemberling</span> &amp; <span>Ted Leo</span>)</li>
Could not parse number for <li>Ep #B2 - <a href="/episode/live-from-sf-sketchfest/">Live from SF Sketchfest!</a> (<span>Doug Benson</span>, <span>Michael Ian Black</span>, <span>Dana Gould</span>, <span>Paul F. Tompkins</span> &amp; <span>Reggie Watts</span>)</li>
Could not parse number for <li>Ep #B1 - <a href="/episode/live-from-vancouver/">Live from Vancouver!</a> (<span>Todd Barry</span>, <span>Nick Thune</span>, <span>Paul F. Tompkins</span>, <span>Garfunkel &amp; Oates</span> &amp; <span>

In [57]:
# Read separate episode sources and merge

earwolf_eps = pd.read_csv("data/earwolf_episodes.csv")
rss_eps = pd.read_csv("data/rss_episodes.csv")

episodes = pd.merge(earwolf_eps, rss_eps, on=["number", "bestOf", "live"], how="inner")

episodes.rename(columns={"guests_x":"guests"}, inplace=True)
del episodes['guests_y']

episodes.to_csv("data/merged_episodes.csv", index=False)

Remove duplicates and fix misspellings

In [56]:
# Count number of appearances per guest

episodes = pd.read_csv("data/merged_episodes_edited.csv")

all_guests = []
for guest_list in episodes["guests"]:
    for guest in json.loads(guest_list):
        all_guests.append(guest)

guest_appearances = pd.Series(name="numberOfAppearances", data=all_guests).value_counts()
guest_appearances.to_csv("data/guests.csv")